In [1]:
import duckdb

base_url = "https://saisonmanager.de"
api_url = base_url + "/api/v2"
leagues_url = api_url + "/leagues.json"
standings_url = api_url + "/leagues/{league_id}/table.json"
scorers_url = api_url + "/leagues/{league_id}/scorer.json"
schedule_url = api_url + "/leagues/{league_id}/schedule.json"
game_url = api_url + "/games/{game_id}.json"

conn = duckdb.connect("floorball-verbandsliga-ost.db")

In [2]:
league_id = 1397

conn.sql(f"""CREATE OR REPLACE TABLE schedule AS SELECT * FROM '{schedule_url.format(league_id=league_id)}';""")
conn.sql(f"""CREATE OR REPLACE TABLE standings AS SELECT * FROM '{standings_url.format(league_id=league_id)}';""")
conn.sql(f"""CREATE OR REPLACE TABLE scorers AS SELECT * FROM '{scorers_url.format(league_id=league_id)}';""")

game_urls = [game_url.format(game_id=game_id) for game_id in conn.sql("SELECT game_id FROM schedule;").df()["game_id"]]
conn.sql(f"""
CREATE OR REPLACE TABLE games AS
    SELECT * FROM read_json_auto([{','.join([f"'{url}'" for url in game_urls])}]);
""")

In [4]:
conn.sql("SELECT team_name, points FROM standings")

┌────────────────────────────┬────────┐
│         team_name          │ points │
│          varchar           │ int64  │
├────────────────────────────┼────────┤
│ Floorball Tigers Magdeburg │     32 │
│ UV Zwigge 07               │     30 │
│ USV Jena                   │     28 │
│ USV Halle Saalebiber II    │     22 │
│ UHC Elster                 │     15 │
│ FBC Phönix Leipzig         │     13 │
│ USV TU Dresden II          │     10 │
│ Red Devils Wernigerode II  │      3 │
└────────────────────────────┴────────┘